### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])[
    'rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [16]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization

    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations

    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''

    # Set up useful values to be used through the rest of the function
    n_users = user_movie_subset.shape[0]  # number of rows in the matrix
    n_movies = user_movie_subset.shape[1]  # number of movies in the matrix
    num_ratings = n_users * n_movies  # total number of ratings in the matrix

    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    # user matrix filled with random values of shape user x latent
    user_mat = np.random.rand(n_users, latent_features)
    # movie matrix filled with random values of shape latent x movies
    movie_mat = np.random.rand(latent_features, n_movies)

    # initialize sse at 0 for first iteration
    sse_accum = 0

    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")

    # for each iteration
    for k in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0

        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):

                # if the rating exists
                if not np.isnan(ratings_mat[i, j]):

                    # compute the error as the actual minus the dot product of the user and movie latent features
                    actual_rating = ratings_mat[i, j]
                    predicted_rating = np.dot(user_mat[i, :], movie_mat[:, j])
                    error = actual_rating - predicted_rating

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error

                    # update the values in each matrix in the direction of the gradient
                    user_mat[i, :] = user_mat[i, :] + \
                        (learning_rate * 2 * error *
                         np.transpose(movie_mat[:, j]))
                    movie_mat[:, j] = movie_mat[:, j] + \
                        (learning_rate * 2 * error *
                         np.transpose(user_mat[i, :]))

        # print results for iteration

    return user_mat, movie_mat

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [21]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(
    ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 


In [22]:
# Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.17933684  8.88633434 10.69287322 10.02489309]
 [ 8.61077579  7.39838645  9.03160167  8.84000272]
 [ 7.96246256  7.03414162  8.27472042  7.64768569]
 [ 9.58922053  8.48487101 10.01324555  9.4576471 ]
 [ 8.41354484  7.41652861  8.96577669  8.60751986]
 [ 6.8289665   5.91655061  7.1077692   6.99822646]
 [ 9.05691638  8.1085805   9.69270691  9.23739614]
 [ 8.20877685  7.0264382   8.56070559  8.32016921]
 [ 8.47441555  7.20506744  8.90571742  8.62113515]
 [ 7.87619005  6.66083144  8.03689889  7.61985594]
 [ 8.80015171  7.7920787   9.22133298  9.00050806]
 [ 9.41109234  8.39355847 10.0175854   9.7431116 ]
 [ 9.31529054  8.13301051  9.69437863  9.28338415]
 [ 6.64739585  6.10021818  6.91449376  6.47785639]
 [ 9.8294869   8.2071505  10.21068317  9.90053939]
 [ 9.88632954  8.76014113 10.21051219  9.54170295]
 [ 8.26889218  7.29898468  8.71837268  8.27733201]
 [ 7.09835622  6.07032816  7.41567694  7.22400355]
 [ 8.26989628  7.20575484  8.74113143  8.54795259]
 [ 8.31352098  7.38566746  8.54

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [23]:
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(
    ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 


In [26]:
# Compare the predicted and actual results
print(np.mean(np.round(np.dot(user_mat, movie_mat)) == ratings_mat))

1.0


**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [27]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [28]:
# run SVD on the matrix with the missing value
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(
    ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 


In [29]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0, 0]))
print()
print("The actual value for the missing rating is {}:".format(
    ratings_mat[0, 0]))
print()
assert np.isnan(preds[0, 0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.254275275696557:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [30]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
# fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(
    first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [44]:
# Replace each of the comments below with the correct values
num_ratings = np.mean(~np.isnan(first_1000_users)) * np.shape(first_1000_users)[
    0] * np.shape(first_1000_users)[1]  # How many actual ratings exist in first_1000_users
# num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = np.shape(first_1000_users)[
    0] * np.shape(first_1000_users)[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.0.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148.0


In [45]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
